In [1]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from prepare_cloth import *
# from anchor_detect import *

# Prepare csv file for labeling

In [3]:
import pickle
import numpy as np
cloth_label_infer = pickle.load(open('../data/cloth/cloth_label_infer.pkl', 'rb'))

In [2]:
attri_name = ['pattern', 'major_color', 'necktie', 'collar', 'scarf', 'sleeve', 'neckline', 'clothing', 'jacket', 'hat', \
              'glass', 'layer', 'necktie_color', 'necktie_pattern', 'hair_color', 'hair_length']
attri_dict = [
        {'solid': 0, 'graphics' : 1, 'striped' : 2, 'floral' : 3, 'plaid' : 4, 'spotted' : 5}, # clothing_pattern
        {'black' : 0, 'white' : 1, 'more color' : 2, 'blue' : 3, 'gray' : 4, 'red' : 5,
                'pink' : 6, 'green' : 7, 'yellow' : 8, 'brown' : 9, 'purple' : 10, 'orange' : 11,
                'cyan' : 12, 'dark blue' : 13}, # major_color
        {'necktie no': 0, 'necktie yes' : 1}, # wearing_necktie
        {'collar no': 0, 'collar yes' : 1}, # collar_presence
        {'scarf no': 0, 'scarf yes' : 1}, # wearing_scarf
        {'long sleeve' : 0, 'short sleeve' : 1, 'no sleeve' : 2}, # sleeve_length
        {'round' : 0, 'folded' : 1, 'v-shape' : 2}, # neckline_shape
        {'shirt' : 0, 'outerwear' : 1, 't-shirt' : 2, 'dress' : 3,
            'tank top' : 4, 'suit' : 5, 'sweater' : 6}, # clothing_category
        {'jacket no': 0, 'jacket yes' : 1}, # wearing_jacket
        {'hat no': 0, 'hat yes' : 1}, # wearing_hat
        {'glasses no': 0, 'glasses yes' : 1}, # wearing_glasses
        {'one layer': 0, 'more layer' : 1}, # multiple_layers
        {'black' : 0, 'white' : 1, 'more color' : 2, 'blue' : 3, 'gray' : 4, 'red' : 5,
                'pink' : 6, 'green' : 7, 'yellow' : 8, 'brown' : 9, 'purple' : 10, 'orange' : 11,
                'cyan' : 12, 'dark blue' : 13}, # necktie_color
        {'solid' : 0, 'striped' : 1, 'spotted' : 2}, # necktie_pattern
        {'black' : 0, 'white': 1, 'blond' : 2, 'brown' : 3, 'gray' : 4}, # hair_color
        {'long' : 0, 'medium' : 1, 'short' : 2, 'bald' : 3} # hair_longth
]
attri_num = len(attri_name)
attri_dict_inv = []
for d in attri_dict:
    d_inv = {}
    for k, v in d.items():
        d_inv[v] = k
    attri_dict_inv.append(d_inv)

In [16]:
## create structure of [[manifest per anchor]] 
pid = -1
current_video = ''
current_anchor = 0
cloth_label_infer_person = []
for p in cloth_label_infer:
    video = p[0]
    aid = p[1]
    if video != current_video or aid != current_anchor:
        cloth_label_infer_person.append([])
        current_video = video
        current_anchor = aid
    cloth_label_infer_person[-1].append(p)

In [17]:
NUM_VOTE = 5
MAX_CLASS = 20
    
fid = 0
images = []
for pid, p_group in enumerate(cloth_label_infer_person):
    if pid % 100 == 0:
        if images != []:
            grid = stitch_img_grid(images, 200, 100, 10)
            filename = '../data/cloth/label/cloth_label_' + str(fid) + '.jpg'
            cv2.imwrite(filename, grid)
            fid += 1
            images = []
        csv = open('../data/cloth/label/cloth_label_' + str(fid) + '.csv', 'w')
    
    video = p_group[0][0]
    votes = []
    num_vote = min(len(p_group), NUM_VOTE)
    for i in range(num_vote):
        votes.append(p_group[i][3])
    votes = np.array(votes).transpose()
    major_vote = []
    for idx, vote in enumerate(votes):
        cnt = np.zeros(MAX_CLASS)
        for v in vote:
            cnt[int(v)] += 1
        major_vote.append(np.argmax(cnt))
    ## write csv
    csv.write(str(pid) + ',' + video + '\n')
    for idx, c in enumerate(major_vote):
        csv.write(attri_dict_inv[idx][c].lower() + ',')
    csv.write('necktie_color,necktie_pattern,hair_color,hair_length\n')
    for i in range(len(p_group)):
        csv.write(str(i))
        if i != len(p_group):
            csv.write(',')
    csv.write('\n')
    ## collect images
    img_dir = '../data/cloth/cloth_label/'
    for i, p in enumerate(p_group):
        img = cv2.imread(img_dir + p[2])
        if i == 0:
            text = str(pid) + '||' + str(i)
        else:
            text = str(i)
        cv2.rectangle(img, (0, 0), (320, 30), color=(255,255,255), thickness=-1)
        cv2.putText(img, text, (0,25), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, color=(0,0,0), thickness=2)
        images.append(img)
    for i in range(20 - len(p_group)):
        images.append(None)
        
csv.close()

# Load csv to create labeled dataset

In [4]:
## create structure of [[manifest per anchor]] 
pid = -1
current_video = ''
current_anchor = 0
cloth_label_infer_person = []
for p in cloth_label_infer:
    video = p[0]
    aid = p[1]
    if video != current_video or aid != current_anchor:
        cloth_label_infer_person.append([])
        current_video = video
        current_anchor = aid
    cloth_label_infer_person[-1].append(p)

In [15]:
import csv
from random import shuffle
labeled_id = [0, 5, 6, 13]
cloth_label = []
for fid in labeled_id:
    csv_name = '../data/cloth/label/cloth_label_c_' + str(fid) + '.csv'
    with open(csv_name) as label_file:
        label_file.seek(0)
        reader = csv.reader(label_file, delimiter=',')
        while True:
            try: 
                head = next(reader)
                attributes = next(reader)
                img_indices = next(reader)
                pid = int(head[0])
                img_indices = [int(idx) for idx in img_indices if idx != '']
                if len(img_indices) == 0:
                    continue
                else:
                    person_group = []
                    for idx in img_indices:
                        manifest = cloth_label_infer_person[pid][idx]
                        true_attribute = []
                        for i, attr in enumerate(attributes):
                            if i == len(attri_name):
                                break
                            if attr == '':
                                true_attribute.append(-1)
                            else:
                                true_attribute.append(attri_dict[i][attr])
                        manifest[-1] = true_attribute
                        person_group.append(manifest)
                    cloth_label.append(person_group)
            except csv.Error:
                print(csv.Error)
                break
            except StopIteration:
                break

shuffle(cloth_label)
# pickle.dump(cloth_label)

In [18]:
# print(len(cloth_label))
# print(cloth_label[0])
pickle.dump(cloth_label, open('../data/cloth/cloth_label_manifest.pkl', 'wb'), protocol=2)